In [1]:
import pandas as pd

In [127]:
df = pd.read_csv("SquidGames.csv", sep='\t')
df = df.drop_duplicates()
df = df.reset_index(drop=True)
df['Tweets'].replace(r"\d+ \d+-\d+-\d+ \d+:\d+:\d+ \+\d+","",regex=True,inplace=True)
df["Author"] = df['Tweets'].str.extract(r'^ <(.*?)>')
df['Tweets'].replace(r"^ <(.*?)>","",regex=True,inplace=True)

In [122]:
df["To"] = df['Tweets'].str.extract(r'^@.*? ')

In [138]:
while True:
    try:
        it =+ 1
        col_name = "To" + str(it)
        print(it)
        df[col_name] = df['Tweets'].str.extract(r'^ (@.*? )*')
        df['Tweets'].replace(r"^ (@.*? )*","",regex=True,inplace=True)
    except:
        break

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [136]:
df

,Tweets,Author,To1
0,Squid games baru nyampek S2 kah?,brucettha,NaN
1,Squid games baru nyampek S2 kah?,brucettha,NaN
2,Squid games sama yumi cells🤩,nirrguna,NaN
3,Yumi's Cells sama squid games,Vitaliatristin3,NaN
4,"Enderman , Steve Human, and Squid games for ...",jason28_hongfu,NaN
...,...,...,...
60321,"Squid Games dekha Aaj pura ka pura,Kasam se Ek...",DesiGamers01,NaN
60322,I just watched ep 6 of squid games ...,KanishkaPal8,NaN
60323,"After watching #squidgamenetflix last night, I...",EmilioCot,NaN
60324,Man I told someone to watch squid games and th...,_ETNOM,NaN
